In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

## Instructions
1. Prepare dataset
2. Load pretrained Tokenizer, call it with dataset -> encoding
3. Build PyTorch Dataset with encodings
4. Load pretrained model
5. Load Trainer and train it (or use native Pytorch training method)

### Data preprocessing

In [ ]:
# Helper funtion to prepare the dataset
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

In [ ]:
# Large Movie review dataset
# https://ai.stanford.edu/~amaas/data/sentiment
train_texts, train_labels = read_imdb_split('data/aclImdb/train')
test_texts, test_labels = read_imdb_split('data/aclImdb/test')

# Create validation set
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[index])
        return item

    def __len__(self):
        return len(self.labels)

### Fine-tune Model with HuggingFace

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [ ]:
# Ensure that all of our sequences are padded to the same length and are truncated
# to be no longer than the model's maximum input length.
# Allows us to feed batches of sequences into the model at the same time.

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, train_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results/",
    num_train_epochs = 2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

In [ ]:
# Initialize model
model = DistilBertForSequenceClassification.from_pretrained(model_name)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
# Fine-tune and train the model
trainer.train()

### Fine-tune Model with PyTorch

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-bert-uncased')
model.to(device)
model.train() # set to training mode

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) # Batch the data

optim = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2

In [ ]:
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward Prop
        outputs = model(input_ids, attention_mask, labels=labels)

        # Calculate gradients
        optim.zero_grad()
        loss = outputs[0]
        loss.backward()

        # Update weights
        optim.step()

model.eval() # Set to eval